# KNIT7 
# Skew Clock Times in NIC
This notebook select one node on the path and disables PTP synchronization at the NIC Clocks and intentionally skews the TIME set in those clocks. 

## Import the FABlib Library

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
import requests

try:
    fablib = fablib_manager()
                     
    fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")

## Get Slice Name

In [ ]:
slice_name=f"Slice for KNIT7 Precision Timing Tutorial"
mySlice = fablib.get_slice(name = slice_name)

## Observe Slice Attributes

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.show()
    slice.list_nodes()
    slice.list_networks()
    slice.list_interfaces()
except Exception as e:
    print(f"Exception: {e}")

## Disable PTP Synchronization and Skew Clock time on NICs Node3

In [ ]:
import datetime
node3 = slice.get_node(name='node3')
for interface in ['ens7' ,'ens8']:
    print (f"Working on node3->{interface}")
    print (f" Stopping PTP Synchronization on node3->{interface}")
    stdout,stderr = node3.execute(f'sudo systemctl stop phc2sys@{interface}.service')
    
    print (f" Get Time from node3->{interface} CLOCK/PHC")
    stdout,stderr = node3.execute("sudo ethtool -T "+interface+"|grep 'PTP Hardware Clock:'|awk '{print $4}'",quiet=True)
    ptp_index = stdout.strip()
    stdout,stderr = node3.execute(f"sudo phc_ctl /dev/ptp{ptp_index} get")
    curr_time = float(stdout.split()[4])
    print (f" Skew Time on node3->{interface} CLOCK/PHC by -5 seconds...")
    stdout,stderr = node3.execute("sudo phc_ctl /dev/ptp"+ptp_index+" set " + str(int(curr_time) - 5) + ";sudo phc_ctl /dev/ptp"+ptp_index+" cmp")

print (f"Time Skew Operation Completed\n\n")